In [ ]:
import os
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

from rdkit import Chem
from rdkit.Chem import AllChem

from molecule_drawer import generate_structure, generate_output_tables, html_output

from IPython.display import SVG
from IPython.core.display import HTML

In [ ]:
pkl_files = glob.glob(f'/Users/nicolairee/Desktop/steno/PhD/git/ReactivityQM/submitit_results/testmol/*result.pkl')


calc_failed = []
df = pd.read_pickle(pkl_files[0])[-1]
for f in pkl_files[1:]:
    d = pd.read_pickle(f)
    if d[0] == 'error':
        calc_failed.append(f)
        continue
    df = df.append(d[-1], ignore_index=True, sort=False)

df.shape[0]

In [ ]:
for idx, row in df.iterrows():

    name = row['name']
    rdkit_smiles = row['smiles']
    rdkit_mols = [Chem.MolFromSmiles(smi) for smi in rdkit_smiles.split('.')]

    elec_sites_list = row['elec_sites']
    elec_names_list = row['elec_names']
    MAA_values_list = row['MAA_values']
    MAA_calc_logs = row['MAA_calc_logs']
    nuc_sites_list = row['nuc_sites']
    nuc_names_list = row['nuc_names']
    MCA_values_list = row['MCA_values']
    MCA_calc_logs = row['MCA_calc_logs']

    result_svg = generate_structure(rdkit_mols, elec_sites_list, MAA_values_list, nuc_sites_list, MCA_values_list, molsPerRow=4)
    
    df_elec = generate_output_tables(rdkit_mols, elec_names_list, MAA_values_list, elec_sites_list, MAA_calc_logs, MAA_or_MCA='MAA')
    df_nuc = generate_output_tables(rdkit_mols, nuc_names_list, MCA_values_list, nuc_sites_list, MCA_calc_logs, MAA_or_MCA='MCA')

    result_output = html_output(rdkit_smiles, result_svg, df_elec, df_nuc)

    with open(f'/Users/nicolairee/Desktop/code2.html', 'w') as f: 
        f.write(result_output)
    
    result_output = result_output.replace("""body {\n                font-family: Arial, sans-serif;\n                background-color: #f7f7f7;\n                margin: 0;\n                padding: 20px;\n                }\n\n                """, """body {\n                font-family: Arial, sans-serif;\n                }\n\n                """)
    result_output = result_output.replace('href="', 'href="../../calculations/') #Change path of "Show File" to make it work from this .ipynb file. OBS! The .html file must be located in "ReactivityQM/calculations"
    display(HTML(result_output))